<a href="https://colab.research.google.com/github/sineii/CS-2565-CSC25/blob/main/Project_Ensemble_500x500edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import pandas as pd
from google.colab import files
from matplotlib import pyplot as plt
from skimage.transform import resize
from skimage.io import imread, imshow
from skimage.util import random_noise
from skimage.restoration import denoise_bilateral
from skimage.restoration import denoise_wavelet
from skimage.restoration import denoise_nl_means
from skimage.restoration import denoise_tv_bregman
from skimage.restoration import denoise_tv_chambolle
from skimage.metrics import peak_signal_noise_ratio, mean_squared_error
import numpy as np
# Structural similarity index
from skimage.metrics import structural_similarity as ssim

In [ ]:
#Add Noise
#Ref https://scikit-image.org/docs/stable/api/skimage.util.html#skimage.util.random_noise

def loadImage(imgNo):
  id = imgfile['fileID'][imgNo]
  imgName = imgfile['fileName'][imgNo]
  filePath = '/content/'+imgName+'.png'
  !gdown $id
  image = imread(filePath, as_gray=True)
  img = resize(image, (500, 500))
  #print('\n',imgName)
  #print(img.shape)
  #imshow(img)
  return img, imgName

def addNoise(originalImg, noise_mode, para):
  #Gaussian #Parameter: var = 0.1, 0.25, 0.5
  #Salt and Pepper #Parameter: amount = 0.1, 0.25, 0.5
  if noise_mode == 'Gaussian':
    imgNoisy = random_noise(originalImg, var=para, mode='gaussian')
  elif noise_mode == 'SaltandPepper':
    imgNoisy = random_noise(originalImg, mode='s&p', salt_vs_pepper=0.5, amount=para)
  return imgNoisy


def run_evaluate(originalImg, denoisedImg):
  mse = mean_squared_error(originalImg, denoisedImg)
  psnr = peak_signal_noise_ratio(originalImg, denoisedImg)
  ssim_const = ssim(originalImg, denoisedImg, data_range=denoisedImg.max() - denoisedImg.min())
  return mse, psnr, ssim_const

def run_bilateral(originalImg, noisyImg):
  denoisedImg = denoise_bilateral(noisyImg, sigma_color=0.05, sigma_spatial=15)
  return run_evaluate(originalImg, denoisedImg)

def run_wavelet(originalImg, noisyImg):
  denoisedImg = denoise_wavelet(noisyImg, sigma=0.1, rescale_sigma=True)
  return run_evaluate(originalImg, denoisedImg)

def run_nlmeans(originalImg, noisyImg):
  denoisedImg = denoise_nl_means(noisyImg, 7, 5, 0.1)
  return run_evaluate(originalImg, denoisedImg)

def run_tvbregman(originalImg, noisyImg):
  denoisedImg = denoise_tv_bregman(noisyImg, weight=5.0, max_num_iter=100, eps=0.001, isotropic=True)
  return run_evaluate(originalImg, denoisedImg)

def run_tvchambolle(originalImg, noisyImg):
  denoisedImg = denoise_tv_chambolle(originalImg, weight=60)
  return run_evaluate(originalImg, denoisedImg)

def run_NL_Wavelet(originalImg, noisyImg):
  denoisedImg_1 = denoise_nl_means(noisyImg, 7, 5, 0.1)
  denoisedImg_2 = denoise_wavelet(noisyImg, sigma=0.1, rescale_sigma=True)
  denoisedImg = (denoisedImg_1 + denoisedImg_2)/2
  return run_evaluate(originalImg, denoisedImg)

def run_NL_TVBregman(originalImg, noisyImg):
  denoisedImg_1 = denoise_nl_means(noisyImg, 7, 5, 0.1)
  denoisedImg_2 = denoise_tv_bregman(originalImg, weight=5.0, max_num_iter=100, eps=0.001, isotropic=True)
  denoisedImg = (denoisedImg_1 + denoisedImg_2)/2
  return run_evaluate(originalImg, denoisedImg)

def run_NL_Bilateral(originalImg, noisyImg):
  denoisedImg_1 = denoise_nl_means(noisyImg, 7, 5, 0.1)
  denoisedImg_2 = denoise_bilateral(noisyImg, sigma_color=0.05, sigma_spatial=15)
  denoisedImg = (denoisedImg_1 + denoisedImg_2)/2
  return run_evaluate(originalImg, denoisedImg)

def run_TVchambolle_TVbregman(originalImg, noisyImg):
  denoisedImg_1 = denoise_tv_chambolle(originalImg, weight=80)
  denoisedImg_2 = denoise_tv_bregman(noisyImg, weight=5.0, max_num_iter=150, eps=0.001, isotropic=True)
  denoisedImg = (denoisedImg_1 + denoisedImg_2)/2
  return run_evaluate(originalImg, denoisedImg) 

def run_TVchambolle_NL(originalImg, noisyImg):
  denoisedImg_1 = denoise_tv_chambolle(originalImg, weight=60)
  denoisedImg_2 = denoise_nl_means(noisyImg, 7, 5, 0.1)
  denoisedImg = (denoisedImg_1 + denoisedImg_2)/2
  return run_evaluate(originalImg, denoisedImg) 

def run_TVchambolle_Wavelet(originalImg, noisyImg):
  denoisedImg_1 = denoise_tv_chambolle(originalImg, weight=60)
  denoisedImg_2 = denoise_wavelet(noisyImg, sigma=0.1, rescale_sigma=True)
  denoisedImg = (denoisedImg_1 + denoisedImg_2)/2
  return run_evaluate(originalImg, denoisedImg) 

def run_TVchambolle_Bilateral(originalImg, noisyImg):
  denoisedImg_1 = denoise_tv_chambolle(originalImg, weight=80)
  denoisedImg_2 = denoise_bilateral(noisyImg, sigma_color=0.05, sigma_spatial=15)
  denoisedImg = (denoisedImg_1 + denoisedImg_2)/2
  return run_evaluate(originalImg, denoisedImg)


def run_TVchambolle_NL_TVBregman(originalImg, noisyImg):
  denoisedImg_1 = denoise_tv_chambolle(noisyImg, weight=80)
  denoisedImg_2 = denoise_nl_means(noisyImg, 7, 5, 0.1)
  denoisedImg_3 = denoise_tv_bregman(noisyImg, weight=5.0, max_num_iter=150, eps=0.001, isotropic=True)
  denoisedImg = (denoisedImg_1 + denoisedImg_2 + denoisedImg_3)/3
  return run_evaluate(originalImg, denoisedImg)  

def run_TVchambolle_NL_Wavelet(originalImg, noisyImg):
  denoisedImg_1 = denoise_tv_chambolle(noisyImg, weight=80)
  denoisedImg_2 = denoise_nl_means(noisyImg, 7, 5, 0.1)
  denoisedImg_3 = denoise_wavelet(noisyImg, sigma=0.1, rescale_sigma=True)
  denoisedImg = (denoisedImg_1 + denoisedImg_2 + denoisedImg_3)/3
  return run_evaluate(originalImg, denoisedImg) 


In [ ]:
fileID = ['1MGcpzCd62ZX7Pm9vG2aaBEfSVQ47WF7G',
      '1eqKCkNL9Psxs44wAvViIcuT-F_VGTh3U',
      '1V-_qJRTFW9OeGWbQS95shJf5o5sUwlDS',
      '1vkjeu0yBweXICZnSpw5QtNEFCL4JM-w5',
      '1UswaVwPmN4U4K8OW5F4APYvtFFEq0BVE',
      '1DQmeI6slXrHThmKlYSyXxy14E-4TxAMg',
      '1Oz1Jt62U7M1fAek_kyAI7t8hiyV4xKwM',
      '1ZLO5yJ9k750H1TTXfCXrKWdmT-Umy64J',
      '1p5PfYGLzJ0-11Mvu3oC5eeABkw1NeCeq']

fileName = ['1 CT Scan ถุงน้ำดี',
      '1 MRI สมองบวม',
      '1 X-ray เอ็กซเรย์ทรวงอกหญิง',
      '2 CT Scan อวัยวะภายในมนุษย์',
      '2 MRI สมองของมนุษย์',
      '2 X-ray นิ้วเท้า',
      '3 CT Scan กระเพาะปัสสาวะ',
      '3 MRI เด็กทารก',
      '3 X-ray ฟัน']

imgfile = pd.DataFrame(list(zip(fileID, fileName)),columns =['fileID', 'fileName'])

imgfile

,fileID,fileName
0,1MGcpzCd62ZX7Pm9vG2aaBEfSVQ47WF7G,1 CT Scan ถุงน้ำดี
1,1eqKCkNL9Psxs44wAvViIcuT-F_VGTh3U,1 MRI สมองบวม
2,1V-_qJRTFW9OeGWbQS95shJf5o5sUwlDS,1 X-ray เอ็กซเรย์ทรวงอกหญิง
3,1vkjeu0yBweXICZnSpw5QtNEFCL4JM-w5,2 CT Scan อวัยวะภายในมนุษย์
4,1UswaVwPmN4U4K8OW5F4APYvtFFEq0BVE,2 MRI สมองของมนุษย์
5,1DQmeI6slXrHThmKlYSyXxy14E-4TxAMg,2 X-ray นิ้วเท้า
6,1Oz1Jt62U7M1fAek_kyAI7t8hiyV4xKwM,3 CT Scan กระเพาะปัสสาวะ
7,1ZLO5yJ9k750H1TTXfCXrKWdmT-Umy64J,3 MRI เด็กทารก
8,1p5PfYGLzJ0-11Mvu3oC5eeABkw1NeCeq,3 X-ray ฟัน


In [ ]:
Image, imgName = loadImage(1) #0-8
noiseType = 'Gaussian' #'SaltandPepper' or 'Gaussian'
noisePara = 0.1  #0.1, 0.25, 0.5

imgFName = imgName+' -500x500- '+noiseType+'-0'+str(noisePara)[2:]
print(imgFName)
nRound = 30

mse_bilateral = []
psnr_bilateral = []
ssim_const_bilateral = []

mse_wavelet = []
psnr_wavelet = []
ssim_const_wavelet = []
mse_nlmeans = []
psnr_nlmeans = []
ssim_const_nlmeans = []
mse_tvbregman = []
psnr_tvbregman = []
ssim_const_tvbregman = []
mse_tvchambolle = []
psnr_tvchambolle = []
ssim_const_tvchambolle = []

mse_NL_Wavelet = []
psnr_NL_Wavelet = []
ssim_const_NL_Wavelet = []

mse_NL_TVBregman = []
psnr_NL_TVBregman = []
ssim_const_NL_TVBregman = []

mse_NL_Bilateral = []
psnr_NL_Bilateral = []
ssim_const_NL_Bilateral = []

mse_TVchambolle_TVbregman = []
psnr_TVchambolle_TVbregman = []
ssim_const_TVchambolle_TVbregman = []

mse_TVchambolle_NL = []
psnr_TVchambolle_NL = []
ssim_const_TVchambolle_NL = []

mse_TVchambolle_Wavelet = []
psnr_TVchambolle_Wavelet = []
ssim_const_TVchambolle_Wavelet =  []

for i in range(nRound):
  print(i,', ', end='')
  imgNoisy = addNoise(originalImg=Image, noise_mode=noiseType, para=noisePara)

  mse, psnr, ssim_const = run_bilateral(Image, imgNoisy)
  mse_bilateral.append(mse)
  psnr_bilateral.append(psnr)
  ssim_const_bilateral.append(ssim_const)

  mse, psnr, ssim_const  = run_wavelet(Image, imgNoisy)
  mse_wavelet.append(mse)
  psnr_wavelet.append(psnr)
  ssim_const_wavelet.append(ssim_const)
  
  mse, psnr, ssim_const  = run_nlmeans(Image, imgNoisy)
  mse_nlmeans.append(mse)
  psnr_nlmeans.append(psnr)
  ssim_const_nlmeans.append(ssim_const)

  mse, psnr, ssim_const  = run_tvbregman(Image, imgNoisy)
  mse_tvbregman.append(mse)
  psnr_tvbregman.append(psnr)
  ssim_const_tvbregman.append(ssim_const)

  mse, psnr, ssim_const  = run_tvchambolle(Image, imgNoisy)
  mse_tvchambolle.append(mse)
  psnr_tvchambolle.append(psnr)
  ssim_const_tvchambolle.append(ssim_const)

  mse, psnr, ssim_const  = run_NL_Wavelet(Image, imgNoisy)
  mse_NL_Wavelet.append(mse)
  psnr_NL_Wavelet.append(psnr)
  ssim_const_NL_Wavelet.append(ssim_const)

  mse, psnr, ssim_const = run_NL_TVBregman(Image, imgNoisy)
  mse_NL_TVBregman.append(mse)
  psnr_NL_TVBregman.append(psnr)
  ssim_const_NL_TVBregman.append(ssim_const)

  mse, psnr, ssim_const  = run_NL_Bilateral(Image, imgNoisy)
  mse_NL_Bilateral.append(mse)
  psnr_NL_Bilateral.append(psnr)
  ssim_const_NL_Bilateral.append(ssim_const)

  mse, psnr, ssim_const = run_TVchambolle_TVbregman(Image, imgNoisy)
  mse_TVchambolle_TVbregman.append(mse)
  psnr_TVchambolle_TVbregman.append(psnr)
  ssim_const_TVchambolle_TVbregman.append(ssim_const)

  mse, psnr, ssim_const  = run_TVchambolle_NL(Image, imgNoisy)
  mse_TVchambolle_NL.append(mse)
  psnr_TVchambolle_NL.append(psnr)
  ssim_const_TVchambolle_NL.append(ssim_const)

  mse, psnr, ssim_const  = run_TVchambolle_Wavelet(Image, imgNoisy)
  mse_TVchambolle_Wavelet.append(mse)
  psnr_TVchambolle_Wavelet.append(psnr)
  ssim_const_TVchambolle_Wavelet.append(ssim_const)


runResults = pd.DataFrame(list(zip(mse_bilateral, psnr_bilateral, ssim_const_bilateral, mse_wavelet, psnr_wavelet, ssim_const_wavelet, mse_nlmeans, psnr_nlmeans, ssim_const_nlmeans, mse_tvbregman, psnr_tvbregman, 
                      ssim_const_tvbregman,mse_tvchambolle, psnr_tvchambolle, ssim_const_tvchambolle, mse_NL_Wavelet, psnr_NL_Wavelet, ssim_const_NL_Wavelet, mse_NL_TVBregman, psnr_NL_TVBregman, ssim_const_NL_TVBregman,
                      mse_NL_Bilateral, psnr_NL_Bilateral, ssim_const_NL_Bilateral, mse_TVchambolle_TVbregman, psnr_TVchambolle_TVbregman, ssim_const_TVchambolle_TVbregman, mse_TVchambolle_NL, psnr_TVchambolle_NL, 
                      ssim_const_TVchambolle_NL, mse_TVchambolle_Wavelet, psnr_TVchambolle_Wavelet, ssim_const_TVchambolle_Wavelet)), 
             columns = ['mse_bilateral', 'psnr_bilateral', 'ssim_bilateral' ,'mse_wavelet', 'psnr_wavelet', 'ssim_wavelet' ,'mse_nlmeans', 'psnr_nlmeans', 'ssim_nlmeans' ,'mse_tvbregman', 'psnr_tvbregman', 
                        'ssim_tvbregman' ,'mse_tvchambolle', 'psnr_tvchambolle','ssim_tvchambolle' ,'mse_NL_Wavelet', 'psnr_NL_Wavelet', 'ssim_NL_Wavelet' ,'mse_NL_TVBregman', 'psnr_NL_TVBregman', 
                        'ssim_NL_TVBregman' ,'mse_NL_Bilateral', 'psnr_NL_Bilateral','ssim_NL_Bilateral','mse_TVchambolle_TVbregman', 'psnr_TVchambolle_TVbregman', 'ssim_TVchambolle_TVbregman','mse_TVchambolle_NL', 
                        'psnr_TVchambolle_NL', 'ssim_TVchambolle_NL', 'mse_TVchambolle_Wavelet' ,'psnr_TVchambolle_Wavelet','ssim_TVchambolle_Wavelet'])


dfrunResults = runResults.T
dfrunResults['Average'] = dfrunResults.mean(axis=1)
display(dfrunResults)
dfrunResults.to_excel(imgFName+'.xlsx') 
files.download(imgFName+'.xlsx')


Downloading...
From: https://drive.google.com/uc?id=1eqKCkNL9Psxs44wAvViIcuT-F_VGTh3U
To: /content/1 MRI สมองบวม.png
100% 505k/505k [00:00<00:00, 120MB/s]
1 MRI สมองบวม -1000x1000- Gaussian-01
0 , 1 , 2 , 3 , 